In [1]:
%autosave 11

Autosaving every 11 seconds


In [2]:
import pandas as pd

Facility_Details=pd.read_excel("Facility Details.xlsx",sheet_name=None,index_col=0)

for i in Facility_Details:
    print(i)

Products
Suppliers
Distribution Centres
Branches
Customers
Vehicle
Commodity Flow Restrictions


In [3]:
K = Facility_Details["Products"].index.tolist()

In [4]:
S = Facility_Details["Suppliers"].index.tolist()

S_Latitude = Facility_Details["Suppliers"]["Latitude"]
S_Longitude = Facility_Details["Suppliers"]["Longitude"]

In [5]:
R = Facility_Details["Distribution Centres"].index.tolist()

R_Latitude = Facility_Details["Distribution Centres"]["Latitude"]
R_Longitude = Facility_Details["Distribution Centres"]["Longitude"]

R_F = Facility_Details["Distribution Centres"]["Fixed Emissions due to Facility Maintenance"]
R_V = Facility_Details["Distribution Centres"]["Variable Emissions due to Daily Operations"]

R_G = Facility_Details["Distribution Centres"]["Fixed Costs due to Facility Maintenance"]
R_W = Facility_Details["Distribution Centres"]["Variable Costs due to Daily Operations"]

In [6]:
B = Facility_Details["Branches"].index.tolist()

B_Latitude = Facility_Details["Branches"]["Latitude"]
B_Longitude = Facility_Details["Branches"]["Longitude"]

B_F = Facility_Details["Branches"]["Fixed Emissions due to Facility Maintenance"]
B_V = Facility_Details["Branches"]["Variable Emissions due to Daily Operations"]

B_G = Facility_Details["Branches"]["Fixed Costs due to Facility Maintenance"]
B_W = Facility_Details["Branches"]["Variable Costs due to Daily Operations"]

In [7]:
V_Emission = pd.concat([B_V, R_V], ignore_index=False, sort=False)
F_Emission = pd.concat([B_F, R_F], ignore_index=False, sort=False)

W_Cost = pd.concat([B_W, R_W], ignore_index=False, sort=False)
G_Cost = pd.concat([B_G, R_G], ignore_index=False, sort=False)

In [8]:
C = Facility_Details["Customers"].index.tolist()

C_Latitude = Facility_Details["Customers"]["Latitude"]
C_Longitude = Facility_Details["Customers"]["Longitude"]

In [9]:
All_Longitudes = {**S_Longitude,**R_Longitude,**B_Longitude,**C_Longitude}
All_Latitudes = {**S_Latitude,**R_Latitude,**B_Latitude,**C_Latitude}

T={}

for source in All_Longitudes:
    for sink in All_Latitudes:
        lat_diff = abs(All_Latitudes[source]-All_Latitudes[sink])
        lon_diff = abs(All_Longitudes[source]-All_Longitudes[sink])
        p=2
        T[source,sink] = ( (lat_diff**p + lon_diff**p)**(1/p) )/1000 #Scaling down distance from meter to kilometers
        
#print(T)
#T["C1","C5"]

In [10]:
E = []

for i in S:
    for j in R+B:
        E.append((i,j))
        
for i in R:
    for j in B:
        E.append((i,j))
        
for i in R+B:
    for j in C:
        E.append((i,j))

In [11]:
V = Facility_Details["Vehicle"].index.tolist()

Q = Facility_Details["Vehicle"]["Capacity (Q)"][V[0]]

F_0 = Facility_Details["Vehicle"]["Fixed Emission (F_0)"][V[0]]
V_0 = Facility_Details["Vehicle"]["Variable Emission (V_0)"][V[0]]

G_0 = Facility_Details["Vehicle"]["Fixed Cost (G_0)"][V[0]]
W_0 = Facility_Details["Vehicle"]["Variable Cost (W_0)"][V[0]]

In [12]:
U_H_D = Facility_Details["Commodity Flow Restrictions"]

In [13]:
M = 1E7

# Data Reading and Set Creation is Complete

In [14]:
#pip install gurobipy
from gurobipy import *

# Set the problem
mdl=Model("Functioning Facility with Emission Reduction")

Set parameter Username
Academic license - for non-commercial use only - expires 2024-12-10


# Variables

In [15]:
x = {}
y = {}
z = {}

In [16]:
for i,j in E:
    
    x[i,j] = mdl.addVar(vtype=GRB.BINARY, name="x%s,%s" % (i,j)) # Equation 10
    
    for k in K:
        y[i,j,k] = mdl.addVar(name="y%s,%s,%s" % (i,j,k)) # Equation 11
            
for i in R+B:
    z[i] = mdl.addVar(vtype=GRB.BINARY, name="z%s" % (i)) # Equation 12

In [17]:
COST = mdl.addVar(name="COST")
EMISSIONS = mdl.addVar(name="EMISSIONS")

# Constraints

In [18]:
mdl.addConstr( quicksum( T[i,j]*( 2*x[i,j]*G_0 + W_0*quicksum(y[i,j,k] for k in K) ) for i,j in E )
              + quicksum( z[i]*G_Cost[i] for i in R+B )
              + quicksum( W_Cost[i]*( quicksum(y[j,i,k] for k in K for j in S) ) for i in R )
              + quicksum( W_Cost[i]*( quicksum(y[i,j,k] for k in K for j in C) ) for i in B )
              == COST)

<gurobi.Constr *Awaiting Model Update*>

In [19]:
mdl.addConstr( quicksum( T[i,j]*( 2*x[i,j]*F_0 + V_0*quicksum(y[i,j,k] for k in K) ) for i,j in E )
              + quicksum( z[i]*F_Emission[i] for i in R+B )
              + quicksum( V_Emission[i]*( quicksum(y[j,i,k] for k in K for j in S) ) for i in R )
              + quicksum( V_Emission[i]*( quicksum(y[i,j,k] for k in K for j in C) ) for i in B )
              == EMISSIONS)

<gurobi.Constr *Awaiting Model Update*>

In [20]:
# Equation 2
for k in K:
    for i in S:
        mdl.addConstr( quicksum( y[i,j,k] for j in R+B ) <= U_H_D[k][i] )

In [21]:
# Equation 3
for k in K:
    for i in R:
        mdl.addConstr( quicksum( y[i,j,k] for j in C+B ) <= quicksum( y[j,i,k] for j in S ) )

In [22]:
# Equation 4
for k in K:
    for i in B:
        mdl.addConstr( quicksum( y[i,j,k] for j in C ) + quicksum( y[j,i,k] for j in S )<= quicksum( y[j,i,k] for j in R ) )

In [23]:
# Equation 5
for k in K:
    for j in R:
        mdl.addConstr( quicksum( y[i,j,k] for i in S ) <= U_H_D[k][j] )

In [24]:
# Equation 6
for k in K:
    for j in B:
        mdl.addConstr( quicksum( y[i,j,k] for i in S+R ) <= U_H_D[k][j] )

In [25]:
# Equation 7
for k in K:
    for j in C:
        mdl.addConstr( quicksum( y[i,j,k] for i in R+B ) == U_H_D[k][j] )

In [26]:
# Equation 8
for i,j in E:
    mdl.addConstr( quicksum( y[i,j,k] for k in K ) <= Q*x[i,j] )

In [27]:
# Equation 9
for i in R+B:
    mdl.addConstr( quicksum(x[j,i] for j in S)
                  +quicksum(x[j,i] for j in R if (not(i in R)))
                  +quicksum(x[i,j] for j in B if (not(i in B)))
                  +quicksum(x[i,j] for j in C)
                  <= M*z[i] )

# EMISSION Objective Function

In [28]:
# Equation 1
mdl.setObjective( EMISSIONS ,GRB.MINIMIZE)

mdl.update()
print(mdl)

mdl.optimize()

import winsound
winsound.Beep(555, 888) # where 555 is the frequency in Hertz and 888 is the duration in miliseconds

<gurobi.Model MIP instance Functioning Facility with Emission Reduction: 177 constrs, 343 vars, Parameter changes: Username=(user-defined)>
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 177 rows, 343 columns and 1657 nonzeros
Model fingerprint: 0x50af1f21
Variable types: 224 continuous, 119 integer (119 binary)
Coefficient statistics:
  Matrix range     [2e-02, 1e+07]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 13 rows and 32 columns
Presolve time: 0.00s
Presolved: 164 rows, 311 columns, 887 nonzeros
Variable types: 202 continuous, 109 integer (109 binary)
Found heuristic solution: objective 649.4977063

Root relaxation: objective 5.984174e+02, 113 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current N

In [29]:
print("Equation 1: Emission Objective")
#print("Cost = ", COST.x)
print("Emissions = ", EMISSIONS.x)
#print("Objective = ", mdl.getObjective().getValue())


VariableCOST_when_objectiveEMISSIONS = COST.x
VariableEMISSIONS_when_objectiveEMISSIONS = EMISSIONS.x


Open_DistributionCentres=0
for i in R:
    Open_DistributionCentres += z[i].x
print("Open Distribution Centres: ",Open_DistributionCentres)

Open_Branches=0
for i in B:
    Open_Branches += z[i].x
print("Open Branches: ",Open_Branches)

Veh_usage=0
for i,j in E:
    Veh_usage += x[i,j].x
print("No. of vehicles used: ",Veh_usage)

Equation 1: Emission Objective
Emissions =  645.2583758797624
Open Distribution Centres:  2.0
Open Branches:  0.0
No. of vehicles used:  14.0


# COST Objective Function

In [30]:
# Equation 13
mdl.setObjective( COST ,GRB.MINIMIZE)

mdl.update()
print(mdl)

mdl.optimize()

import winsound
winsound.Beep(555, 888) # where 555 is the frequency in Hertz and 888 is the duration in miliseconds

<gurobi.Model MIP instance Functioning Facility with Emission Reduction: 177 constrs, 343 vars, Parameter changes: Username=(user-defined)>
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 177 rows, 343 columns and 1657 nonzeros
Model fingerprint: 0xfc0a84f5
Variable types: 224 continuous, 119 integer (119 binary)
Coefficient statistics:
  Matrix range     [2e-02, 1e+07]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]

MIP start from previous solve produced solution with objective 676.455 (0.05s)
Loaded MIP start from previous solve with objective 676.455

Presolve removed 13 rows and 32 columns
Presolve time: 0.00s
Presolved: 164 rows, 311 columns, 887 nonzeros
Variable types: 202 continuous, 109 integer (109 binary)

Root relaxation: object

In [31]:
print("Equation 13: Cost Objective")
#print("Cost = ", COST.x)
print("Emissions = ", EMISSIONS.x)
#print("Objective = ", mdl.getObjective().getValue())


VariableCOST_when_objectiveCOST = COST.x
VariableEMISSIONS_when_objectiveCOST = EMISSIONS.x


Open_DistributionCentres=0
for i in R:
    Open_DistributionCentres += z[i].x
print("Open Distribution Centres: ",Open_DistributionCentres)

Open_Branches=0
for i in B:
    Open_Branches += z[i].x
print("Open Branches: ",Open_Branches)

Veh_usage=0
for i,j in E:
    Veh_usage += x[i,j].x
print("No. of vehicles used: ",Veh_usage)

Equation 13: Cost Objective
Emissions =  692.9829684246297
Open Distribution Centres:  2.0
Open Branches:  0.0
No. of vehicles used:  12.0


# %-age increase in Emissions when minimizing Cost

In [32]:
Percent_age = (VariableEMISSIONS_when_objectiveCOST - VariableEMISSIONS_when_objectiveEMISSIONS) / VariableEMISSIONS_when_objectiveEMISSIONS

print(Percent_age*100," % ")

print("Vehicles Available: ",len(E))

7.396198845121274  % 
Vehicles Available:  111
